<h1>Capstone Project - Predicting Suitable location for a New Italian Restaurant throughout the city of Newyork<h2>

<h1>The Battle of Neighborhoods</h1>  
<h3>Introduction/Business Problem </h3> 
<br> 
The United States of America is one of the most diversified countries in the world. This is because it is one of the most developed countries in the world and many people from different countries flock there. These make it possible to see different cultures all cohabiting together. As a result of this, many different businesses have also sprouted up. There are now chinese restaurants, African restaurants, Indian restaurants, etc.
<br>
The craving for food from different  countries has also risen. For instance, a chinese may want to try out how Indian food tastes, an american may also want to try Afican delicacy, etc. Although there are a lot of restaurants in New York, the project focuses on the suitable location of a new Indian restaurant in Queens, New York. The stakeholders will determine the best location to plant a new Indian restaurant in Queens, New York.Some important characteristics of this location is that it has no Indian restaurants within 400m.
<br>
This project aims to predict where to set up a new Indian restaurant in the neighborhood of Queens where there is none within 400 metres. Other parameters are used to determine the precise position on which to concentrate. Stakeholders will be involved in forecasting precisely where a new Indian restaurant would be situated for competitive advantage and business values. Others who'll be involved include Indians in that locality.

<h5>Introduction: Problem Statement</h5>

The United States of America is one of the most diversified countries in the world. This is because it is one of the most developed countries in the world and many people from different countries flock there. These make it possible to see different cultures all cohabiting together. 

As a result of this factor and many others, many different businesses have also sprouted up. There are now chinese restaurants, African restaurants, Indian restaurants, etc. 

The craving for food from different  countries has also risen. For instance, a Chinese may want to try out how Indian food tastes, an american may also want to try out Afican delicacy, etc. Although there are a lot of restaurants in New York, the project focuses on the suitable location of a new Italian restaurant in Queens, New York. The stakeholders will determine the best location to plant a new Italian restaurant in cities in Newyork, United States.

<h5>Data</h5>
 
In order to answer the above questions, data on New York City neighborhoods, boroughs to include boundaries, latitude, longitude, restaurants, and restaurant ratings and tips are required.

New York City data containing the neighborhoods and boroughs, latitudes, and longitudes will be obtained from the data source: https://cocl.us/new_york_dataset

All data related to locations and quality of Italian restaurants will be obtained via the FourSquare API utilized via the Request library in Python.


<h6>Methodology</h6>

Data will be collected from https://cocl.us/new_york_dataset and cleaned and processed into a dataframe.

FourSquare be used to locate all venues and then filtered by Japanese restaurants. Ratings, tips, and likes by users will be counted and added to the dataframe.

Data will be sorted based on rankings.

Finally, the data be will be visually assessed using graphing from Python libraries.

<b>IMPORT ALL REQUIRED LIBRARIES</b><br>

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests

#from bs4 import BeautifulSoup
import os

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline

import seaborn as sns

print('Libraries imported.')

Solving environment: - 

<b>MY CREDENTIALS, client ID, secret and Version Number from Foursquare </b><br>

In [3]:
# Define Foursquare Credentials and Version

CLIENT_ID = 'HL3CBGYWYOVNGY1V2ZFV2ZBYXYWVGM5NCDVQXWCD0KNGVABT' # your Foursquare ID
CLIENT_SECRET = 'XM4OKH34C01SM4G1PPVMY4FHII3CZZPEBP1TVLJTGDVXNGUR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


<b>MY FUNCTIONS DECLARATION AND DEFINITION</b><br>

In [4]:

def geo_location(address):
    # get geo location of address
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return latitude,longitude


def get_venues(lat,lng):
    #set variables
    radius=400
    LIMIT=100
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    # get all the data
    results = requests.get(url).json()
    venue_data=results['response']['groups'][0]['items']
    venue_details=[]
    for row in venue_data:
        try:
            venue_id=row['venue']['id']
            venue_name=row['venue']['name']
            venue_category=row['venue']['categories'][0]['name']
            venue_details.append([venue_id,venue_name,venue_category])
        except KeyError:
            pass
    column_names=['ID','Name','Category']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df


def get_venue_details(venue_id):
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            venue_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
    # get all the data
    results = requests.get(url).json()
    print(results)
    venue_data=results['response']['venue']
    venue_details=[]
    try:
        venue_id=venue_data['id']
        venue_name=venue_data['name']
        venue_likes=venue_data['likes']['count']
        venue_rating=venue_data['rating']
        venue_tips=venue_data['tips']['count']
        venue_details.append([venue_id,venue_name,venue_likes,venue_rating,venue_tips])
    except KeyError:
        pass
    column_names=['ID','Name','Likes','Rating','Tips']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df


def get_new_york_data():
    url='https://cocl.us/new_york_dataset'
    resp=requests.get(url).json()
    # all data is present in features label
    features=resp['features']
    # define the dataframe columns
    column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
    # instantiate the dataframe
    new_york_data = pd.DataFrame(columns=column_names)
    for data in features:
        borough = data['properties']['borough'] 
        neighborhood_name = data['properties']['name']
        neighborhood_latlon = data['geometry']['coordinates']
        neighborhood_lat = neighborhood_latlon[1]
        neighborhood_lon = neighborhood_latlon[0]
        new_york_data = new_york_data.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    return new_york_data


<b>NewYork City Data</b><br>

In [7]:
ny_data = get_new_york_data()
ny_data.head()

NameError: name 'requests' is not defined

In [8]:
ny_data.shape

NameError: name 'ny_data' is not defined

<b>DATA ANALYSIS</b><br>

In [ ]:
clr = "green"
ny_data.groupby('Borough')['Neighborhood'].count().plot.bar(figsize=(10,5), color=clr)
plt.title('Neighborhoods per Borough: NYC', fontsize = 20)
plt.xlabel('Borough', fontsize = 15)
plt.ylabel('No. Neighborhoods',fontsize = 15)
plt.xticks(rotation = 'horizontal')
plt.legend()
plt.show()

<b><h3>Number of Italian restaurants there are in each neighborhood and Borough</h3> </b>

In [ ]:
# queens has most neighborhoods
# prepare neighborhood list that contains japanese restaurants
column_names=['Borough', 'Neighborhood', 'ID','Name']
italian_rest_ny=pd.DataFrame(columns=column_names)
count=1
for row in ny_data.values.tolist():
    Borough, Neighborhood, Latitude, Longitude=row
    venues = get_venues(Latitude,Longitude)
    italian_restaurants=venues[venues['Category']=='Italian Restaurant']   
    print('(',count,'/',len(ny_data),')','Italian Restaurants in '+Neighborhood+', '+Borough+':'+str(len(italian_restaurants)))
    print(row)
    for resturant_detail in italian_restaurants.values.tolist():
        id, name , category=resturant_detail
        italian_rest_ny = italian_rest_ny.append({'Borough': Borough,
                                                'Neighborhood': Neighborhood, 
                                                'ID': id,
                                                'Name' : name
                                               }, ignore_index=True)
    count+=1

In [ ]:
italian_rest_ny.to_csv('italian_rest_ny_tocsv1.csv')

In [ ]:
italian_ny = pd.read_csv('italian_rest_ny_tocsv1.csv')
italian_rest_ny.tail()

In [ ]:
italian_rest_ny.shape

We got 230 Italian Restaurants across the New York City. Manhattan has the highest number of Italian restaurants.

In [ ]:
italian_rest_ny.groupby('Borough')['ID'].count().plot.bar(figsize=(10,5), color=clr)
plt.title('Italian Restaurants per Borough: NYC', fontsize = 20)
plt.xlabel('Borough', fontsize = 15)
plt.ylabel('No. of Italian Restaurants', fontsize=15)
plt.xticks(rotation = 'horizontal')
plt.legend()
plt.show()

In [ ]:
NOofNeigh = 6 # top number for graphing all the same past 6
italian_rest_ny.groupby('Neighborhood')['ID'].count().nlargest(NOofNeigh).plot.bar(figsize=(10,5), color=clr)
plt.title('Italian Restaurants per Neighborhood: NYC', fontsize = 20)
plt.xlabel('Neighborhood', fontsize = 15)
plt.ylabel('No.of Italian Restaurants', fontsize=15)
plt.xticks(rotation = 'horizontal')
plt.legend()
plt.show()

In [ ]:
italian_rest_ny[italian_rest_ny['Neighborhood']=='Belmont']